In [1]:
###########################################
PROJECT_NAME = 'square_bar'
###########################################

In [2]:
import config #place your Openai key here
from openai import OpenAI

# Initialize the OpenAI client with API key
client = OpenAI(api_key=config.OPENAI_API_KEY)

In [3]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None
model_call_count = 0

# Compose the system message once and append to history
with open("system_sif.txt", 'r') as file:
    sif_message = file.read().strip()
with open("square_bar.geo", 'r') as file:
    geometry_message = file.read().strip()

system_message = sif_message + geometry_message 
    
conversation_history.append({"role": "system", "content": system_message})

In [4]:
#FUNCTIONS FOR INTERACTING WITH CHATBOT
    
def chat_with_bot(user_messages):
    global last_response
    # Add the new user input to the conversation history
    conversation_history.append({"role": "user", "content": user_messages})
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=conversation_history,
        temperature=0
    )
    # Extract the model's response
    model_response = response.choices[0].message.content
    # Update the last response
    last_response = model_response
    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    # Print the model's response
    return print(model_response)

#Saves .geo file from response
def extract_and_save_sif_file(response_text): 
    file_path = f"{PROJECT_NAME}.sif"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_SIF"
        end_marker = f"//END_SIF"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [5]:
import os

def generate_ELMER_mesh():
    try:
        current_directory = os.getcwd()
        msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        !ElmerGrid 14 2 "{msh_file}" -autoclean
        print(f"ELMER project created in {PROJECT_NAME}")
        print(f"Copy .sif file in {PROJECT_NAME}")
        
    except ValueError as e:
        print(f"ERROR.")

In [6]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."), 
# when you are satisfied save the .sif file with
# extract_and_save_sif_file(last_response) and
# generate ELMER project with generate_ELMER_mesh()
# use Paraview for visualization
###############################################

In [7]:
chat_with_bot("""Create a sif file for a 'linear elasticity' simulation of a steel bar with the geometry in the geo file. 
Boundary 1 is fixed (no deformation), while Boudary 2 is subjected of a Force of 100000000 N in the y direction
""")

To create the .sif file for a 'linear elasticity' simulation of a steel bar, we need to define the simulation type, constants, body properties, solver configuration, equation settings, material properties, initial conditions, and boundary conditions. Since the simulation type is not explicitly mentioned, I will assume it is a "Steady state" simulation. If this is incorrect, please provide the correct simulation type.

Here is the .sif file content based on the provided information:

```plaintext
//BEGIN_SIF

Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 1
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.670374419e-08
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 

In [8]:
chat_with_bot("""
1. Simulation Type: "Steady state"
2. Constants: do not include gravity
3. Material Properties: use typical values for steel
4. Boundary Conditions: "Boundary 1" is Bottom and "Boundary 2" the "Top" in the .geo file
              """)

Based on the updated requirements, here is the revised .sif file content for a 'Steady state' linear elasticity simulation of a steel bar without gravity, using typical values for steel, and with the specified boundary conditions for the "Bottom" and "Top" surfaces as defined in the .geo file:

```plaintext
//BEGIN_SIF

Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 1
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Stefan Boltzmann = 5.670374419e-08
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Boltzmann Constant = 1.380649e-23
  Unit Charge = 1.6021766e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "SteelBar"
  Equation = 1
  Material = 1
  Initial condition = 1
End

Solver 1
  Equation = Lin

In [9]:
extract_and_save_sif_file(last_response)

square_bar.sif file saved


In [10]:
generate_ELMER_mesh()


Starting program Elmergrid
Elmergrid reading in-line arguments
Lower dimensional boundaries will be removed
Materials and boundaries will be renumbered
Nodes that do not appear in any element will be removed
Output will be saved to file C:\Users\alexiada-admin\Documents\Alessio\microarticle\simulation\square_bar\square_bar.

Elmergrid loading data:
-----------------------
Format chosen using the first line: $MeshFormat
Gmsh version is 4.1
Loading mesh in Gmsh format 4.1 from file C:\Users\alexiada-admin\Documents\Alessio\microarticle\simulation\square_bar\square_bar.msh
Reading 8 entities in 0D
Reading 12 entities in 1D
Reading 6 entities in 2D
Reading 1 entities in 3D
Allocating for 10250 knots and 57462 elements.
Boundary name for physical group 1 is: Bottom
Boundary name for physical group 2 is: Top
Boundary name for physical group 3 is: Side1
Boundary name for physical group 4 is: Side2
Boundary name for physical group 5 is: Side3
Boundary name for physical group 6 is: Side4
Body 

In [11]:
new_directory = os.path.join(os.getcwd(), f"{PROJECT_NAME}")
os.chdir(new_directory)

In [12]:
!ElmerSolver square_bar.sif

ELMER SOLVER (v 9.0) STARTED AT: 2023/11/30 22:17:30
ParCommInit:  Initialize #PEs:            1
MAIN: 
MAIN: =============================================================
MAIN: ElmerSolver finite element software, Welcome!
MAIN: This program is free software licensed under (L)GPL
MAIN: Copyright 1st April 1995 - , CSC - IT Center for Science Ltd.
MAIN: Webpage http://www.csc.fi/elmer, Email elmeradm@csc.fi
MAIN: Version: 9.0 (Rev: Release, Compiled: 2022-08-02)
MAIN:  Running one task without MPI parallelization.
MAIN:  Running with just one thread per task.
MAIN:  Lua interpreter linked in.
MAIN: =============================================================
MAIN: 
MAIN: 
MAIN: -------------------------------------
MAIN: Reading Model: square_bar.sif
LoadInputFile: Scanning input file: square_bar.sif
LoadInputFile: Scanning only size info
LoadInputFile: First time visiting
LoadInputFile: Reading base load of sif file
LoadInputFile: Loading input file: square_bar.sif
LoadInputFile: Rea